# 챕터 5: 출력 포맷팅과 Claude를 위한 발화

- [레슨](#lesson)
- [연습문제](#exercises)
- [예제 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정하세요.

In [ ]:
!pip install anthropic

# 파이썬의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY & MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# 프리필 텍스트를 위한 새로운 인수 추가, 기본값은 빈 문자열
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 레슨

**Claude는 다양한 방식으로 출력을 포맷할 수 있습니다**. 그렇게 해달라고 요청하기만 하면 됩니다!

이러한 방법 중 하나는 XML 태그를 사용하여 응답을 다른 불필요한 텍스트로부터 분리하는 것입니다. 여러분은 이미 XML 태그를 사용하여 프롬프트를 Claude에게 더 명확하고 파싱하기 쉽게 만들 수 있다는 것을 배웠습니다. 또한 **Claude에게 XML 태그를 사용하여 출력을 인간에게 더 명확하고 쉽게 이해할 수 있도록 만들어달라고 요청할 수도 있습니다**.

### 예시

챕터 2에서 Claude에게 서문을 완전히 건너뛰도록 요청하여 해결한 '시 서문 문제'를 기억하시나요? **Claude에게 시를 XML 태그에 넣어달라고 말함으로써** 유사한 결과를 달성할 수도 있습니다.

In [ ]:
# 변수 내용
ANIMAL = "토끼"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

왜 이런 것을 하고 싶을까요? 음, **XML 태그에 출력을 넣으면 최종 사용자가 XML 태그 사이의 내용을 추출하는 짧은 프로그램을 작성하여 시와 시만을 안정적으로 얻을 수 있습니다**.

이 기법의 확장은 **첫 번째 XML 태그를 `assistant` 턴에 넣는 것입니다. `assistant` 턴에 텍스트를 넣으면, 기본적으로 Claude에게 Claude가 이미 무언가를 말했고, 그 지점부터 계속해야 한다고 말하는 것입니다. 이 기법을 "Claude를 위한 발화" 또는 "Claude의 응답 미리 채우기"라고 합니다.

아래에서 우리는 첫 번째 `<haiku>` XML 태그로 이를 수행했습니다. Claude가 우리가 중단한 지점에서 바로 계속하는 것을 주목하세요.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답을 위한 프리필
PREFILL = "<haiku>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴:")
print(PROMPT)
print("\n어시스턴트 턴:")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude는 또한 다른 출력 포맷팅 스타일, 특히 `JSON`을 사용하는 데 뛰어납니다. JSON 출력을 강제하고 싶다면(결정적이지는 않지만 그에 가깝게), Claude의 응답을 여는 괄호 `{`로 미리 채울 수도 있습니다.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. \"첫째_줄\", \"둘째_줄\", \"셋째_줄\"을 키로 하는 JSON 형식을 사용하세요."

# Claude의 응답을 위한 프리필
PREFILL = "{"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

아래는 **동일한 프롬프트에서 여러 입력 변수와 출력 포맷팅 지정이 모두 XML 태그를 사용하여 수행되는** 예시입니다.

In [ ]:
# 첫 번째 입력 변수
EMAIL = "안녕 잭, 네가 작성하기로 한 프롬프트에 대한 간단한 업데이트를 요청하려고 해."

# 두 번째 입력 변수
ADJECTIVE = "고전 영어식"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"안녕 Claude. 여기 이메일이 있어: <email>{EMAIL}</email>. 이 이메일을 더 {ADJECTIVE}으로 만들어줘. 새 버전을 <{ADJECTIVE}_email> XML 태그에 써줘."

# Claude의 응답을 위한 프리필 (이제 변수가 있는 f-string으로)
PREFILL = f"<{ADJECTIVE}_email>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 보너스 레슨

API를 통해 Claude를 호출하는 경우, 닫는 XML 태그를 `stop_sequences` 매개변수에 전달하여 Claude가 원하는 태그를 출력하면 샘플링을 중지하도록 할 수 있습니다. 이는 Claude가 이미 여러분이 관심 있는 답변을 제공한 후 결론적인 발언을 제거함으로써 비용과 마지막 토큰까지의 시간을 절약할 수 있습니다.

위의 내용을 변경하지 않고 레슨 프롬프트를 실험해보고 싶다면, 레슨 노트북의 맨 아래로 스크롤하여 [**예제 플레이그라운드**](#example-playground)를 방문하세요.

---

## 연습문제
- [연습문제 5.1 - 스테프 커리 GOAT](#exercise-51---steph-curry-goat)
- [연습문제 5.2 - 두 개의 하이쿠](#exercise-52---two-haikus)
- [연습문제 5.3 - 두 개의 하이쿠, 두 마리 동물](#exercise-53---two-haikus-two-animals)

### 연습문제 5.1 - 스테프 커리 GOAT
선택을 강요받으면 Claude는 마이클 조던을 역대 최고의 농구 선수로 지목합니다. Claude가 다른 사람을 선택하도록 할 수 있을까요?

`PREFILL` 변수를 변경하여 **Claude가 역대 최고의 농구 선수가 스테프 커리라는 상세한 논증을 하도록 강요하세요**. 이 연습문제의 초점이므로 `PREFILL` 외에는 아무것도 변경하지 마세요.

In [ ]:
# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"역대 최고의 농구 선수는 누구인가요? 특정한 선수 한 명을 선택해 주세요."

# Claude의 응답을 위한 프리필
PREFILL = ""

# Claude의 응답 받기
response = get_completion(PROMPT, prefill=PREFILL)

# 연습문제 정답 채점 함수
def grade_exercise(text):
    return bool(re.search("워리어", text))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 연습문제 5.2 - 두 개의 하이쿠
Claude가 하나가 아닌 동물에 대한 두 개의 하이쿠를 작성하도록 XML 태그를 사용하여 아래의 `PROMPT`를 수정하세요. 한 시가 끝나고 다른 시가 시작하는 지점이 명확해야 합니다.

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답을 위한 프리필
PREFILL = "<haiku>"

# Claude의 응답 받기
response = get_completion(PROMPT, prefill=PREFILL)

# 연습문제 정답 채점 함수
def grade_exercise(text):
    return bool(
        (re.search("고양이", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 연습문제 5.3 - 두 개의 하이쿠, 두 마리 동물
**Claude가 두 마리의 다른 동물에 대한 두 개의 하이쿠를 생성하도록** 아래의 `PROMPT`를 수정하세요. 첫 번째 대체를 위한 대체자로 `{ANIMAL1}`을, 두 번째 대체를 위한 대체자로 `{ANIMAL2}`를 사용하세요.

In [ ]:
# 첫 번째 입력 변수
ANIMAL1 = "고양이"

# 두 번째 입력 변수
ANIMAL2 = "개"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL1}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답 받기
response = get_completion(PROMPT)

# 연습문제 정답 채점 함수
def grade_exercise(text):
    return bool(re.search("꼬리", text.lower()) and re.search("고양이", text.lower()) and re.search("<haiku>", text))

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(response)
print("\n------------------------------------------ 채점 ------------------------------------------")
print("이 연습문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### 축하합니다!

지금까지 모든 연습문제를 해결했다면, 다음 챕터로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!

---

## 예제 플레이그라운드

이곳은 이 레슨에서 보여준 프롬프트 예제들을 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답에 어떤 영향을 미치는지 볼 수 있는 공간입니다.

In [ ]:
# 변수 내용
ANIMAL = "토끼"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. <haiku> 태그에 넣어주세요."

# Claude의 응답을 위한 프리필
PREFILL = "<haiku>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴:")
print(PROMPT)
print("\n어시스턴트 턴:")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 변수 내용
ANIMAL = "고양이"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"{ANIMAL}에 대한 하이쿠를 써주세요. \"첫째_줄\", \"둘째_줄\", \"셋째_줄\"을 키로 하는 JSON 형식을 사용하세요."

# Claude의 응답을 위한 프리필
PREFILL = "{"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 첫 번째 입력 변수
EMAIL = "안녕 잭, 네가 작성하기로 한 프롬프트에 대한 간단한 업데이트를 요청하려고 해."

# 두 번째 입력 변수
ADJECTIVE = "고전 영어식"

# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = f"안녕 Claude. 여기 이메일이 있어: <email>{EMAIL}</email>. 이 이메일을 더 {ADJECTIVE}으로 만들어줘. 새 버전을 <{ADJECTIVE}_email> XML 태그에 써줘."

# Claude의 응답을 위한 프리필 (이제 변수가 있는 f-string으로)
PREFILL = f"<{ADJECTIVE}_email>"

# Claude의 응답 출력
print("--------------------------- 변수 대체가 포함된 전체 프롬프트 ---------------------------")
print("사용자 턴")
print(PROMPT)
print("\n어시스턴트 턴")
print(PREFILL)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))